In [1]:
import numpy as np

In [2]:
import torch
import time

# Set the matrix size
matrix_size = 1000

# Generate two random matrices
A = torch.randn(matrix_size, matrix_size)
B = torch.randn(matrix_size, matrix_size)

# Function to perform matrix multiplication on CPU
def cpu_matrix_multiplication(A, B):
    # Ensure tensors are on the CPU
    A = A.to('cpu')
    B = B.to('cpu')
    
    start_time = time.time()
    result = torch.mm(A, B)
    end_time = time.time()
    
    return end_time - start_time, result

# Function to perform matrix multiplication on GPU
def gpu_matrix_multiplication(A, B):
    # Ensure tensors are on the GPU
    A = A.to('mps')  # Use 'mps' for Metal Performance Shaders on M1/M2
    B = B.to('mps')
    
    # Warm-up to ensure accurate timing
    torch.mm(A, B)
    
    # Perform matrix multiplication
    start_time = time.time()
    result = torch.mm(A, B)
    end_time = time.time()
    
    return end_time - start_time, result

# Perform the matrix multiplication on CPU
cpu_time, res_cpu = cpu_matrix_multiplication(A, B)
print(f"CPU Time: {cpu_time:.6f} seconds")

# Perform the matrix multiplication on GPU
gpu_time, res_gpu = gpu_matrix_multiplication(A, B)
print(f"GPU Time (using MPS): {gpu_time:.6f} seconds")

# Compare the results
speedup = cpu_time / gpu_time
print(f"Speedup using GPU (MPS): {speedup:.2f}x")



CPU Time: 0.011946 seconds
GPU Time (using MPS): 0.000243 seconds
Speedup using GPU (MPS): 49.17x


# forward

In [3]:
X1=np.array([[1,2,3],[4,5,6]])
W1=np.array([[6,7],[3,3],[2,2]])
Y1=X1@W1
f=lambda x: x+1
df=lambda x: np.ones(x.shape)
y_out=f(Y1)
L=np.sum(y_out,axis=1)

In [4]:
X1

array([[1, 2, 3],
       [4, 5, 6]])

In [5]:
W1

array([[6, 7],
       [3, 3],
       [2, 2]])

In [6]:
Y1

array([[18, 19],
       [51, 55]])

In [7]:
y_out

array([[19, 20],
       [52, 56]])

In [8]:
L

array([ 39, 108])

# backward

In [9]:
dL_dyout=np.ones((2,2))
dyout_Y1=df(Y1)
Y1_W1=X1.T


In [10]:
dL_dyout

array([[1., 1.],
       [1., 1.]])

In [11]:
dyout_Y1

array([[1., 1.],
       [1., 1.]])

In [12]:
dyout_Y1

array([[1., 1.],
       [1., 1.]])

In [13]:
dyout_Y1.shape[1]

2

In [14]:
func=torch.optim.Adam.__init__
adam_params = func.__code__.co_varnames[:func.__code__.co_argcount]
adam_params

('self', 'params', 'lr', 'betas', 'eps', 'weight_decay', 'amsgrad')

In [15]:
hparams={"lr": 1, "a":2}
parsed_arguments={ key: value for key,value in hparams.items() if key in adam_params}
parsed_arguments


{'lr': 1}

In [16]:
from itertools import product

grid_search_spaces={"a":[1,3], "b":[20,25]}
configs = []


    # More general implementation using itertools
for instance in product(*grid_search_spaces.values()):
    configs.append(dict(zip(grid_search_spaces.keys(), instance)))

configs

[{'a': 1, 'b': 20}, {'a': 1, 'b': 25}, {'a': 3, 'b': 20}, {'a': 3, 'b': 25}]

In [17]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class ShortcutModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(ShortcutModel, self).__init__()
        # Linear layer
        self.fc1 = nn.Linear(input_size, hidden_size)
        
        # Batch normalization layer
        self.bn = nn.BatchNorm1d(hidden_size)
        
        # Output layer
        self.fc2 = nn.Linear(hidden_size, output_size)
        
        ## combinde
        self.ffs=nn.Sequential(self.fc1,self.bn,nn.ReLU())

        # Shortcut connection (identity mapping)
        if input_size != hidden_size:
            # If input and output dimensions are different, use a linear layer to match dimensions
            self.shortcut = nn.Linear(input_size, hidden_size)
        else:
            # Identity mapping if dimensions match
            self.shortcut = nn.Identity()

    def forward(self, x):
        # First linear transformation
        #out = self.fc1(x)
        
        # Batch normalization and activation
        #out = self.bn(out)
        #out = F.relu(out)
        
        # Shortcut connection (adding the input to the transformed output)
        shortcut = self.shortcut(x)
        out =self.ffs(x)
        out += shortcut
        
        # Pass through output layer
        out = self.fc2(out)
        
        return out

# Example usage
input_size = 10
hidden_size = 20
output_size = 1
model = ShortcutModel(input_size, hidden_size, output_size)

# Example input tensor
x = torch.randn(5, input_size)
output = model(x)

print(output)
print(sum(p.numel() for p in model.parameters() if p.requires_grad))

tensor([[ 3.0587],
        [ 0.3325],
        [-0.5717],
        [ 0.5167],
        [-0.1494]], grad_fn=<AddmmBackward0>)
501


In [18]:
model

ShortcutModel(
  (fc1): Linear(in_features=10, out_features=20, bias=True)
  (bn): BatchNorm1d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=20, out_features=1, bias=True)
  (ffs): Sequential(
    (0): Linear(in_features=10, out_features=20, bias=True)
    (1): BatchNorm1d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (shortcut): Linear(in_features=10, out_features=20, bias=True)
)

In [19]:
hparams["optimizer"]="Adam"
getattr(torch.optim, hparams["optimizer"].capitalize())

torch.optim.adam.Adam

In [20]:
getattr(torch.nn, "CrossEntropyLoss")

torch.nn.modules.loss.CrossEntropyLoss

In [21]:
import numpy as np

In [22]:
x=np.ones((1,31))

In [23]:
np.squeeze(x)

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [24]:
x=np.arange(10).reshape((2,5))

In [25]:
sample_mean = np.mean(x, axis=0)

x_minus_mean = x - sample_mean[np.newaxis,:]

In [26]:
x_minus_mean

array([[-2.5, -2.5, -2.5, -2.5, -2.5],
       [ 2.5,  2.5,  2.5,  2.5,  2.5]])

In [27]:
import torch.nn as nn
num_conv_layer=4
conv_layers=[nn.Conv2d(in_channels=2**(3+num_conv_layer-k), out_channels=2**(4+num_conv_layer-k), \
                               kernel_size=(k+1,k+1)) for k in range(num_conv_layer-1)]

In [28]:
conv_layers[-1]


Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))

In [29]:
import numpy as np
x = np.array([[1,2],[3,4]])

In [30]:
x.flatten()

array([1, 2, 3, 4])

In [32]:
from torchvision.models import resnet18, ResNet18_Weights
import torch
model=resnet18(weights=ResNet18_Weights.DEFAULT)

In [33]:
x=torch.randn((1,3,240,240))
model.to("mps")
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [37]:
x0=torch.randn((1,3,224,224)).to("mps")

In [38]:
c1=model.conv1 
y1=c1(x0)
b1=model.bn1 
y2=b1(y1)
relu1=model.relu 
y3=b1(y2)
maxpool1=model.maxpool 
y4=maxpool1(y3)

In [39]:
print(x0.shape)
print(y1.shape)
print(y2.shape)
print(y3.shape)
print(y4.shape)



torch.Size([1, 3, 224, 224])
torch.Size([1, 64, 112, 112])
torch.Size([1, 64, 112, 112])
torch.Size([1, 64, 112, 112])
torch.Size([1, 64, 56, 56])


In [ ]:
l1=model.layer1
x1=torch.randn((1,64,240,240)).to("mps")
l2=model.layer2
x2=torch.randn((1,64,240,240)).to("mps")

l3=model.layer3
x3=torch.randn((1,128,120,120)).to("mps")
l4=model.layer4
x4=torch.randn((1,256,60,60)).to("mps")
avgpool=model.avgpool
x5=torch.randn((1,512,30,30)).to("mps")
with torch.no_grad():
    print(l1(x1).shape)
    print(l2(x2).shape)
    print(l3(x3).shape)
    print(l4(x4).shape)
    print(avgpool(x5).shape)
    model.forward

In [ ]:
model.fc = torch.nn.Linear(model.fc.in_features, 23)

In [32]:
x=torch.randn((1,3,240,240))
model.to("mps")
model.eval()
with torch.no_grad():
    model.forward(x.to("mps"))

In [41]:
import numpy as np 
  
# declaring an array 
a = [[1, 2, 3], [4, 5, 6], [7, 8, 9]] 
  
# use the repeat function to upsample the array 
print(np.repeat(a, 5, axis=1)) 


[[1 1 1 1 1 2 2 2 2 2 3 3 3 3 3]
 [4 4 4 4 4 5 5 5 5 5 6 6 6 6 6]
 [7 7 7 7 7 8 8 8 8 8 9 9 9 9 9]]


In [42]:
class ResNet18(nn.Module):
    def __init__(self, n_classes):
        super(ResNet18, self).__init__()
        
        self.dropout_percentage = 0.5
        self.relu = nn.ReLU()
        
        # BLOCK-1 (starting block) input=(224x224) output=(56x56)
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=(7,7), stride=(2,2), padding=(3,3))
        self.batchnorm1 = nn.BatchNorm2d(64)
        self.maxpool1 = nn.MaxPool2d(kernel_size=(3,3), stride=(2,2), padding=(1,1))
        
        # BLOCK-2 (1) input=(56x56) output = (56x56)
        self.conv2_1_1 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=(3,3), stride=(1,1), padding=(1,1))
        self.batchnorm2_1_1 = nn.BatchNorm2d(64)
        self.conv2_1_2 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=(3,3), stride=(1,1), padding=(1,1))
        self.batchnorm2_1_2 = nn.BatchNorm2d(64)
        self.dropout2_1 = nn.Dropout(p=self.dropout_percentage)
        # BLOCK-2 (2)
        self.conv2_2_1 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=(3,3), stride=(1,1), padding=(1,1))
        self.batchnorm2_2_1 = nn.BatchNorm2d(64)
        self.conv2_2_2 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=(3,3), stride=(1,1), padding=(1,1))
        self.batchnorm2_2_2 = nn.BatchNorm2d(64)
        self.dropout2_2 = nn.Dropout(p=self.dropout_percentage)
        
        # BLOCK-3 (1) input=(56x56) output = (28x28)
        self.conv3_1_1 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=(3,3), stride=(2,2), padding=(1,1))
        self.batchnorm3_1_1 = nn.BatchNorm2d(128)
        self.conv3_1_2 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=(3,3), stride=(1,1), padding=(1,1))
        self.batchnorm3_1_2 = nn.BatchNorm2d(128)
        self.concat_adjust_3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=(1,1), stride=(2,2), padding=(0,0))
        self.dropout3_1 = nn.Dropout(p=self.dropout_percentage)
        # BLOCK-3 (2)
        self.conv3_2_1 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=(3,3), stride=(1,1), padding=(1,1))
        self.batchnorm3_2_1 = nn.BatchNorm2d(128)
        self.conv3_2_2 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=(3,3), stride=(1,1), padding=(1,1))
        self.batchnorm3_2_2 = nn.BatchNorm2d(128)
        self.dropout3_2 = nn.Dropout(p=self.dropout_percentage)
        
        # BLOCK-4 (1) input=(28x28) output = (14x14)
        self.conv4_1_1 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=(3,3), stride=(2,2), padding=(1,1))
        self.batchnorm4_1_1 = nn.BatchNorm2d(256)
        self.conv4_1_2 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=(3,3), stride=(1,1), padding=(1,1))
        self.batchnorm4_1_2 = nn.BatchNorm2d(256)
        self.concat_adjust_4 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=(1,1), stride=(2,2), padding=(0,0))
        self.dropout4_1 = nn.Dropout(p=self.dropout_percentage)
        # BLOCK-4 (2)
        self.conv4_2_1 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=(3,3), stride=(1,1), padding=(1,1))
        self.batchnorm4_2_1 = nn.BatchNorm2d(256)
        self.conv4_2_2 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=(3,3), stride=(1,1), padding=(1,1))
        self.batchnorm4_2_2 = nn.BatchNorm2d(256)
        self.dropout4_2 = nn.Dropout(p=self.dropout_percentage)
        
        # BLOCK-5 (1) input=(14x14) output = (7x7)
        self.conv5_1_1 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=(3,3), stride=(2,2), padding=(1,1))
        self.batchnorm5_1_1 = nn.BatchNorm2d(512)
        self.conv5_1_2 = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=(3,3), stride=(1,1), padding=(1,1))
        self.batchnorm5_1_2 = nn.BatchNorm2d(512)
        self.concat_adjust_5 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=(1,1), stride=(2,2), padding=(0,0))
        self.dropout5_1 = nn.Dropout(p=self.dropout_percentage)
        # BLOCK-5 (2)
        self.conv5_2_1 = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=(3,3), stride=(1,1), padding=(1,1))
        self.batchnorm5_2_1 = nn.BatchNorm2d(512)
        self.conv5_2_2 = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=(3,3), stride=(1,1), padding=(1,1))
        self.batchnorm5_2_2 = nn.BatchNorm2d(512)
        self.dropout5_2 = nn.Dropout(p=self.dropout_percentage)
        
        # Final Block input=(7x7) 
        self.avgpool = nn.AvgPool2d(kernel_size=(7,7), stride=(1,1))
        self.fc = nn.Linear(in_features=1*1*512, out_features=1000)
        self.out = nn.Linear(in_features=1000, out_features=n_classes)
        # END
    
    def forward(self, x):
        
        # block 1 --> Starting block
        x = self.relu(self.batchnorm1(self.conv1(x)))
        op1 = self.maxpool1(x)
        
        
        # block2 - 1
        x = self.relu(self.batchnorm2_1_1(self.conv2_1_1(op1)))    # conv2_1 
        x = self.batchnorm2_1_2(self.conv2_1_2(x))                 # conv2_1
        x = self.dropout2_1(x)
        # block2 - Adjust - No adjust in this layer as dimensions are already same
        # block2 - Concatenate 1
        op2_1 = self.relu(x + op1)
        # block2 - 2
        x = self.relu(self.batchnorm2_2_1(self.conv2_2_1(op2_1)))  # conv2_2 
        x = self.batchnorm2_2_2(self.conv2_2_2(x))                 # conv2_2
        x = self.dropout2_2(x)
        # op - block2
        op2 = self.relu(x + op2_1)
    
        
        # block3 - 1[Convolution block]
        x = self.relu(self.batchnorm3_1_1(self.conv3_1_1(op2)))    # conv3_1
        x = self.batchnorm3_1_2(self.conv3_1_2(x))                 # conv3_1
        x = self.dropout3_1(x)
        # block3 - Adjust
        op2 = self.concat_adjust_3(op2) # SKIP CONNECTION
        # block3 - Concatenate 1
        op3_1 = self.relu(x + op2)
        # block3 - 2[Identity Block]
        x = self.relu(self.batchnorm3_2_1(self.conv3_2_1(op3_1)))  # conv3_2
        x = self.batchnorm3_2_2(self.conv3_2_2(x))                 # conv3_2 
        x = self.dropout3_2(x)
        # op - block3
        op3 = self.relu(x + op3_1)
        
        
        # block4 - 1[Convolition block]
        x = self.relu(self.batchnorm4_1_1(self.conv4_1_1(op3)))    # conv4_1
        x = self.batchnorm4_1_2(self.conv4_1_2(x))                 # conv4_1
        x = self.dropout4_1(x)
        # block4 - Adjust
        op3 = self.concat_adjust_4(op3) # SKIP CONNECTION
        # block4 - Concatenate 1
        op4_1 = self.relu(x + op3)
        # block4 - 2[Identity Block]
        x = self.relu(self.batchnorm4_2_1(self.conv4_2_1(op4_1)))  # conv4_2
        x = self.batchnorm4_2_2(self.conv4_2_2(x))                 # conv4_2
        x = self.dropout4_2(x)
        # op - block4
        op4 = self.relu(x + op4_1)

        
        # block5 - 1[Convolution Block]
        x = self.relu(self.batchnorm5_1_1(self.conv5_1_1(op4)))    # conv5_1
        x = self.batchnorm5_1_2(self.conv5_1_2(x))                 # conv5_1
        x = self.dropout5_1(x)
        # block5 - Adjust
        op4 = self.concat_adjust_5(op4) # SKIP CONNECTION
        # block5 - Concatenate 1
        op5_1 = self.relu(x + op4)
        # block5 - 2[Identity Block]
        x = self.relu(self.batchnorm5_2_1(self.conv5_2_1(op5_1)))  # conv5_2
        x = self.batchnorm5_2_1(self.conv5_2_1(x))                 # conv5_2
        x = self.dropout5_2(x)
        # op - block5
        op5 = self.relu(x + op5_1)


        # FINAL BLOCK - classifier 
        x = self.avgpool(op5)
        x = x.reshape(x.shape[0], -1)
        x = self.relu(self.fc(x))
        x = self.out(x)

        return x

In [44]:
model=ResNet18(n_classes=10)


In [46]:
count=0
for parameter in model.parameters():
    count+=len(parameter.reshape((-1)))
print(count)

11702530


In [49]:
h=224

h=np.floor((h-1)/2)+1
np.floor((h-1)/2)+1

56.0